<a href="https://colab.research.google.com/github/martynna/dw_matrix_road_signs/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [10]:
import os
import datetime

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

#a tu do zwizualizowania krzywej uczenia wczytujemy tensorboard
%load_ext tensorboard 

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_3/dw_matrix_road_signs'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_3/dw_matrix_road_signs


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, Y_train = train['features'], train['labels']
X_test, Y_test = test['features'], test['labels']

In [0]:
if Y_train.ndim == 1: Y_train = to_categorical(Y_train)
if Y_test.ndim == 1: Y_test = to_categorical(Y_test)

input_shape = X_train.shape[1:] #czyli bierzemy wymiary zdjecia i kanaly RGB
num_classes = Y_train.shape[1] #czyli ilosc kolumn (rodzajow znakow w tym przypadku)

In [0]:
def train_model(model, X_train, Y_train, params_fit={}): #params_fit nie jest obowiazkowy, ma domyslne wartosci zainicjowane ponizej
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])

  #sciezka do katalogu w ktorym beda logi:
  logdir=os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S") )
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      Y_train,
      batch_size = params_fit.get('batch_size', 128), #wielkosc paczek przekazywanych sieci do uczenia sie
      epochs = params_fit.get('epochs', 5), #potrzebna do okreslenia ile razy trzeba wstrzyknac batcha zeby byla 1 iteracje (czyli zeby wszystkie dane uczace przepuscic przez model)
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, Y_train)), #aby obserwowac na ile model sie przyuczyl, po danej iteracji
      callbacks = [tensorboard_callback]
      )
  
  return(model)

def predict(model_trained, X_array, Y_array, scoring = accuracy_score):
  y_test_norm = np.argmax(Y_array, axis=1)

  y_predict_prob = model_trained.predict(X_array)
  y_pred = np.argmax(y_predict_prob, axis=1) #wartosc sprognozowana

  return accuracy_score(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape), #siec konwolucyjna
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding = 'same'),
                    MaxPool2D(),
                    Dropout(0.3),

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'), #siec konwolucyjna - regula podwajania filtrow w stosunku do poprzedniej warstwy
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3), #zeby model sie nie przeuczyl/ przyuczyl zbyt szybko

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'), #padding - dodajemy zera do obrazka, zeby nie zmniejszal sie jego wymiar z kazda kolejna warstwa
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Flatten(),
                     
                    Dense(1024, activation='relu'),
                    Dropout(0.3),
                     
                    Dense(1024, activation='relu'),
                    Dropout(0.3),

                    Dense(num_classes, activation='softmax'),
])

In [7]:
model = get_cnn_v5(input_shape, num_classes)

trained_mod = train_model(model, X_train, Y_train)
predict(trained_mod, X_test, Y_test)


Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 2.7917 - accuracy: 0.2425 - val_loss: 1.5604 - val_accuracy: 0.4708
Epoch 2/5
272/272 [==============================] - 5s 18ms/step - loss: 1.0845 - accuracy: 0.6494 - val_loss: 0.3151 - val_accuracy: 0.9035
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.4688 - accuracy: 0.8520 - val_loss: 0.1062 - val_accuracy: 0.9685
Epoch 4/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2794 - accuracy: 0.9134 - val_loss: 0.0586 - val_accuracy: 0.9822
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2163 - accuracy: 0.9335 - val_loss: 0.0587 - val_accuracy: 0.9821


0.9492063492063492

In [0]:
def get_model(params):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape), #siec konwolucyjna
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding = 'same'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_one']),

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'), #siec konwolucyjna - regula podwajania filtrow w stosunku do poprzedniej warstwy
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_two']), #zeby model sie nie przeuczyl/ przyuczyl zbyt szybko

                    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding = 'same'), #padding - dodajemy zera do obrazka, zeby nie zmniejszal sie jego wymiar z kazda kolejna warstwa
                    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_three']),
                     
                    Flatten(),
                     
                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_dense_block_one']),
                     
                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_dense_block_two']),

                    Dense(num_classes, activation='softmax'),
])

In [0]:
def func_obj(params): #pudlo logiczne, black-box
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])

  model.fit(
      X_train, 
      Y_train,
      batch_size = int(params.get('batch_size', 128)), #wielkosc paczek przekazywanych sieci do uczenia sie
      epochs = 5, #potrzebna do okreslenia ile razy trzeba wstrzyknac batcha, zeby byla 1 iteracje (czyli zeby wszystkie dane uczace przepuscic przez model)
      verbose = 0,
      )
  
  score = model.evaluate(X_test, Y_test, verbose=0)
  accuracy=score[1]
  print(params, 'accuracy{}'.format(params))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model } #-accuracy, bo dazymy do minimum wiec maksymalny accuracy chcemy przerzucic na ujemna, zeby bylo najmniejsze

In [27]:
space = {
    'batch_size' : hp.quniform('batch_size', 50, 200, 20), #od 50 do 200 z krokiem 20
    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one', 0.2, 0.7),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two', 0.2, 0.7),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three', 0.2, 0.7),

    'dropout_dense_block_one' : hp.uniform('dropout_dense_block_one', 0.2, 0.7),
    'dropout_dense_block_two' : hp.uniform('dropout_dense_block_two', 0.2, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.4545404789134465, 'dropout_cnn_block_three': 0.6018913196081208, 'dropout_cnn_block_two': 0.3939717610019611, 'dropout_dense_block_one': 0.3728274797037089, 'dropout_dense_block_two': 0.3291796172531746}
accuracy{'batch_size': 100.0, 'dropout_cnn_block_one': 0.4545404789134465, 'dropout_cnn_block_three': 0.6018913196081208, 'dropout_cnn_block_two': 0.3939717610019611, 'dropout_dense_block_one': 0.3728274797037089, 'dropout_dense_block_two': 0.3291796172531746}
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.2848670232926903, 'dropout_cnn_block_three': 0.23643379607212228, 'dropout_cnn_block_two': 0.20269452397647264, 'dropout_dense_block_one': 0.48787838020131863, 'dropout_dense_block_two': 0.6982010061555056}
accuracy{'batch_size': 180.0, 'dropout_cnn_block_one': 0.2848670232926903, 'dropout_cnn_block_three': 0.23643379607212228, 'dropout_cnn_block_two': 0.20269452397647264, 'dropout_dense_block_one': 0.48787838020131863, 'dropout_dens